In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import os
import time
import json

from eval_utils import filter_boxes, calculate_iou, calculate_precision_recall

import pandas as pd
from rpn import _transforms as T

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_transform(image_enhancement="FALSE"):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    if image_enhancement == "TRUE":
        transforms.append(T.ContrastBasedAdaptiveGammaCorrection())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

#### Read and Print the evaluation config from the JSON file

In [3]:
#Create a folder with unix timestamp
root_path = os.getcwd() + '/results/' + str(int(time.time())) + '/'
os.mkdir(root_path)

with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)


print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_france/all_dataset/",
 "DATASET": "FRANCE",
 "MODEL": "SAT-SAM",
 "IMAGE_ENHANCEMENT": "FALSE",
 "FILTRATION": "TRUE",
 "TRAIN_TYPE": "FINETUNE",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_OVERLAP_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.3,
 "RPN_MODEL_PATH": "rpn/checkpoints/1692042915/rpn_model_1.07.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset_test = PlanetscopeDataset(eval_config['DATASET_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))

torch.manual_seed(1)
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-int(len(indices)*0.2):])

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE'])
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)
elif eval_config['MODEL'] == 'SAM':
    vanilla_sam_model = pipeline("mask-generation", model="facebook/sam-vit-large", device=device)
elif eval_config['MODEL'] == 'MASKRCNN':
    maskrcnn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE']) #Load the Pre-Trained MaskRCNN model

In [6]:
results = pd.DataFrame(columns=['parcel_id', 'parcel_path', 'gt_mask_ct', 'pred_mask_ct', 'mean_iou', 'p_50', 'r_50', 'p_70', 'r_70', 'p_90', 'r_90'])

for i, (id, sam_image, rpn_image, target, ensemble, path)  in enumerate(dataset_test): 
    try:
        if eval_config['MODEL'] == 'SAT-SAM':
            print('SAT-SAM')
            rpn_image = rpn_image.squeeze(0).to(device)  
            predictions = rpn_model.predict(rpn_image)
            predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            
            if eval_config['FILTRATION'] == 'TRUE':
                filtered_predictions = filter_boxes(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
                low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
            else:
                low_res_masks, iou_predictions = sam_model.predict(sam_image, predictions['boxes'])
            
            high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
            pred_masks = high_res_masks.squeeze().cpu().numpy()
        
        elif eval_config['MODEL'] == 'SAM':
            print('SAM')
            outputs = vanilla_sam_model(sam_image, points_per_batch=32)
            pred_masks = outputs["masks"]
        
        elif eval_config['MODEL'] == 'MASKRCNN':
            print('MASKRCNN')
            rpn_image = rpn_image.squeeze(0).to(device)
            predictions = maskrcnn_model.predict(rpn_image)
            predictions = maskrcnn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
            pred_masks = predictions['masks']

        iou_score, iou_matrix = calculate_iou(target_masks=np.array(target['masks']), predicted_masks=np.array(pred_masks))
        
        p_50, r_50 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.5)
        p_70, r_70 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.7)
        p_90, r_90 = calculate_precision_recall(iou_matrix, len(pred_masks), len(target['masks']), threshold=0.9)
        
        # Round off the values to 2 decimal places
        iou_score = round(iou_score, 2)
        p_50, r_50 = round(p_50, 2), round(r_50, 2)
        p_70, r_70 = round(p_70, 2), round(r_70, 2)
        p_90, r_90 = round(p_90, 2), round(r_90, 2)

        print("Image Id: ", id, " Average IoU Score: ", iou_score)
        print("IoU Matrix: ", iou_matrix)
        print("Precision: ", p_50, p_70, p_90)
        print("Recall: ", r_50, r_70, r_90)

        results.loc[i] = [id, path, len(target['masks']), len(pred_masks), iou_score, p_50, r_50, p_70, r_70, p_90, r_90]
    except:
        print("Error in image: ", id)
        continue

SAT-SAM
num_target_masks:  29
num_predicted_masks:  15
row_ind:  [ 0  1  2  3  4  5  9 10 11 15 18 20 21 26 28]
col_ind:  [14 12  9  6 13  7  4  0 10  1  3  8 11  2  5]
Image Id:  2204  Average IoU Score:  0.57
IoU Matrix:  [0.         0.04082935 0.         0.71332019 0.         0.91421105
 0.65742964 0.91774953 0.6        0.90627406 0.7775828  0.91186359
 0.69338266 0.91851266 0.47365789]
Precision:  0.67 0.47 0.33
Recall:  0.34 0.24 0.17
SAT-SAM
num_target_masks:  15
num_predicted_masks:  13
row_ind:  [ 0  1  2  3  4  5  6  7  8  9 10 11 14]
col_ind:  [10  2  6  9  8 12  0  4  3  7 11  1  5]
Image Id:  1762  Average IoU Score:  0.35
IoU Matrix:  [0.47355711 0.         0.         0.         0.         0.01856946
 0.85401674 0.94261982 0.49443846 0.32445471 0.         0.57714671
 0.86721424]
Precision:  0.31 0.23 0.08
Recall:  0.27 0.2 0.07
SAT-SAM
num_target_masks:  8
num_predicted_masks:  11
row_ind:  [0 1 2 3 4 5 6 7]
col_ind:  [4 2 0 3 1 5 8 6]
Image Id:  1368  Average IoU Score:  

In [ ]:
mean_IoU = results['mean_iou'].mean()
eval_config['MEAN_IOU'] = mean_IoU

In [ ]:
results.to_csv(root_path + 'results', index=False)

# save model and configuration
with open(root_path + 'eval_config.json', 'w') as f:
    json.dump(eval_config, f, indent=1)
    f.close()
print("Evaluation Configuration saved to " + root_path + 'train_config.json') 